<a href="https://colab.research.google.com/github/hzang41981/ESC-50/blob/master/HW3P2/HW3P2_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11785 HW3P2: Automatic Speech Recognition

Welcome to HW3P2. In this homework, you will be using the same data from HW1 but will be incorporating sequence models. We recommend you get familaried with sequential data and the working of RNNs, LSTMs and GRUs to have a smooth learning in this part of the homework.

Disclaimer: This starter notebook will not be as elaborate as that of HW1P2 or HW2P2. You will need to do most of the implementation in this notebook because, it is expected after 2 HWs, you will be in a position to write a notebook from scratch. You are welcomed to reuse the code from the previous starter notebooks but may also need to make appropriate changes for this homework. <br>
We have also given you 3 log files for the Very Low Cutoff (Levenshtein Distance = 30) so that you can observe how loss decreases.

Common errors which you may face


*   Shape errors: Half of the errors from this homework will account to this category. Try printing the shapes between intermediate steps to debug
*   CUDA out of Memory: When your architecture has a lot of parameters, this can happen. Golden keys for this is, (1) Reducing batch_size (2) Call *torch.cuda.empty_cache* often, even inside your training loop, (3) Call *gc.collect* if it helps and (4) Restart run time if nothing works







# Prelimilaries

You will need to install packages for decoding and calculating the Levenshtein distance

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX # We also install a summary package to check our model's forward before training

     |████████████████████████████████| 50 kB 4.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149852 sha256=5e56d7139c893b13c3e271a75199d6a91cea5a1d0e215e0bd24b78220f87446f
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1102 (delta 16), reused 28 (delta 13), pack-reused 1063
Receiving objects: 100% (1102/1102), 780.91 KiB | 8.87 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_p

# Libraries

In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle (TODO)

You need to set up your Kaggle and download the data

In [4]:
import json

TOKEN = {"username":"hzang41981","key":"07ce2b8c32b2a27182316ae7fe638b80"}

! pip install kaggle==1.5.12
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content

     |████████████████████████████████| 58 kB 4.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=f00924399ee277d28c975b6181b89c20177c54692f4136bbbf33022dce9fbaa2
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json
- path is now set to: /content


In [5]:
! kaggle competitions download -c 11-785-s22-hw3p2
! unzip competitions/11-785-s22-hw3p2/11-785-s22-hw3p2.zip

Streaming output truncated to the last 5000 lines.
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-020.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-021.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-022.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-023.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-024.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-025.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-026.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-027.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-028.npy  
  inflating: hw3p2_student_data/hw3p2_student_data/train/transcript/7113-086041-029.npy  
  inflating: hw3p2_student_data/hw3p2_student_dat

# Dataset and dataloading (TODO)

In [6]:
# PHONEME_MAP is the list that maps the phoneme to a single character. 
# The dataset contains a list of phonemes but you need to map them to their corresponding characters to calculate the Levenshtein Distance
# You final submission should not have the phonemes but the mapped string
# No TODOs in this cell

G_PHONEME_MAP = [
    " ", #BLANK
    ".", #SIL
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #HH
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]

G_PHONEMES = ["", 'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH']

In [18]:
import csv
# This cell is where your actual TODOs start
# You will need to implement the Dataset class by your own. You may also implement it similar to HW1P2 (dont require context)
# The steps for implementation given below are how we have implemented it.
# However, you are welcomed to do it your own way if it is more comfortable or efficient. 

class LibriSamples(torch.utils.data.Dataset):

    def __init__(self, data_path, partition= "train"): # You can use partition to specify train or dev
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"

        self.X_files = os.listdir(self.X_dir)
        self.Y_files = os.listdir(self.Y_dir)

        # TODO: store PHONEMES from phonemes.py inside the class. phonemes.py will be downloaded from kaggle.
        # You may wish to store PHONEMES as a class attribute or a global variable as well.
        self.PHONEMES = G_PHONEMES

        assert(len(self.X_files) == len(self.Y_files))

    def __len__(self):
        return len(self.X_files)

    def __getitem__(self, ind):
    
        X_path = self.X_dir + self.X_files[ind]
        Y_path = self.Y_dir + self.Y_files[ind]
        X = np.load(X_path)
        X = torch.tensor((X - X.mean(axis=0))/X.std(axis=0))
        Y = [self.PHONEMES.index(yy) for yy in np.load(Y_path)[1:-1]]

        # Remember, the transcripts are a sequence of phonemes. Eg. np.array(['<sos>', 'B', 'IH', 'K', 'SH', 'AA', '<eos>'])
        # You need to convert these into a sequence of Long tensors
        # Tip: You may need to use self.PHONEMES
        # Remember, PHONEMES or PHONEME_MAP do not have '<sos>' or '<eos>' but the transcripts have them. 
        # You need to remove '<sos>' and '<eos>' from the trancripts. 
        # Inefficient way is to use a for loop for this. Efficient way is to think that '<sos>' occurs at the start and '<eos>' occurs at the end.
        
        Yy = torch.tensor(Y)

        return X, Yy
    
    def collate_fn(batch):

        batch_x = [x for x,y in batch]
        batch_y = [y for x,y in batch]


        batch_x_pad = pad_sequence(batch_x, batch_first=True,)
        lengths_x = [x.size(dim=0) for x,y in batch]

        batch_y_pad = pad_sequence(batch_y, batch_first=True,)
        lengths_y = [y.size(dim = 0) for x,y in batch]

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)


# You can either try to combine test data in the previous class or write a new Dataset class for test data
class LibriSamplesTest(torch.utils.data.Dataset):

    def __init__(self, data_path, test_order): # test_order is the csv similar to what you used in hw1

        self.X_dir = data_path + "/" + 'test' + "/mfcc/"
        test_order_list = self.parse_csv(data_path + '/' + test_order)
        self.X_names = test_order_list

    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[0])
        return subset[1:]
    def __len__(self):
        return len(self.X_names)
    
    def __getitem__(self, ind):
        X_path = self.X_dir + self.X_names[ind]
        X = np.load(X_path)
        X = torch.tensor((X - X.mean(axis=0))/X.std(axis=0))

        return X
    
    def collate_fn(batch):
        batch_x = [x for x in batch]        
        batch_x_pad = pad_sequence(batch_x, batch_first=True,)
        lengths_x = [x.size(dim=0) for x in batch]

        return batch_x_pad, torch.tensor(lengths_x)

In [19]:
batch_size = 64

root = '/content/hw3p2_student_data/hw3p2_student_data'

train_data = LibriSamples(root, 'train')
val_data = LibriSamples(root, 'dev')
test_data = LibriSamplesTest(root, 'test/test_order.csv')

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, collate_fn=LibriSamples.collate_fn)
val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, collate_fn=LibriSamples.collate_fn)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, collate_fn=LibriSamplesTest.collate_fn)

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


# Model Configuration (TODO)

In [20]:
class Network(nn.Module):

    def __init__(self): # You can add any extra arguments as you wish

        super(Network, self).__init__()

        embedding_dim = 256
        lstm_units = 256
        lstm_layers = 4
        bidirectional = True
        num_classes = 41
        dropout = 0.5
        hidden_dim = 2048
        hidden_dim_2 = 1024
        kernel_size_1 = 7
        kernel_size_2 = 3
        kernel_size_3 = 1

        # Embedding layer converts the raw input into features which may (or may not) help the LSTM to learn better 
        # For the very low cut-off you dont require an embedding layer. You can pass the input directly to the  LSTM
        self.embedding = nn.Sequential(
            nn.Conv1d(13, 13, kernel_size_1, padding=kernel_size_1//2, groups=13), # 56 , stride = 2
            nn.BatchNorm1d(13),
            nn.GELU(),
            nn.Conv1d(13, 64, kernel_size_2, padding=kernel_size_2//2),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Conv1d(64, embedding_dim, kernel_size_3), # 56
            nn.Dropout(p=dropout)
            )
        
        self.lstm = nn.LSTM(embedding_dim,
                            lstm_units,
                            num_layers=lstm_layers,
                            bidirectional=bidirectional,
                            batch_first=True, dropout=0.1)
        
        num_directions = 2 if bidirectional else 1

        self.classification = nn.Sequential(nn.Linear(num_directions * lstm_units, hidden_dim),
                                            nn.Dropout(p=dropout),
                                            nn.Linear(hidden_dim, hidden_dim_2),
                                            nn.Dropout(p=dropout),
                                            nn.Linear(hidden_dim_2, num_classes))
        
        
        # TODO: # Create a single layer, uni-directional LSTM with hidden_size = 256
        # Use nn.LSTM() Make sure that you give in the proper arguments as given in https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

    def forward(self, x, lx): # TODO: You need to pass atleast 1 more parameter apart from self and x

        #kernel_size_1 = 13
        emb = self.embedding(x.permute(0,2,1))

        # x is returned from the dataloader. So it is assumed to be padded with the help of the collate_fn
        # packed_input = pack_padded_sequence(emb.permute(0,2,1),(lx - kernel_size_1 + 12) // 2 ,batch_first=True,enforce_sorted=False)
        packed_input = pack_padded_sequence(emb.permute(0,2,1),lx,batch_first=True,enforce_sorted=False)
        # TODO: Pack the input with pack_padded_sequence. Look at the parameters it requires

        out1, (out2, out3) = self.lstm(packed_input)
        # TODO: Pass packed input to self.lstm
        # As you may see from the LSTM docs, LSTM returns 3 vectors. Which one do you need to pass to the next function?
        out, lengths  = pad_packed_sequence(out1, batch_first=True)
        # TODO: Need to 'unpack' the LSTM output using pad_packed_sequence

        out = self.classification(out)
        # TODO: Pass unpacked LSTM output to the classification layer
        out = F.log_softmax(out,2)
        # Optional: Do log softmax on the output. Which dimension?

        return out, lengths # TODO: Need to return 2 variables

model = Network().to(device)
print(model)
#summary(model, x.to(device), lx) # x and lx are from the previous cell

Network(
  (embedding): Sequential(
    (0): Conv1d(13, 13, kernel_size=(7,), stride=(1,), padding=(3,), groups=13)
    (1): BatchNorm1d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU()
    (3): Conv1d(13, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): GELU()
    (6): Conv1d(64, 256, kernel_size=(1,), stride=(1,))
    (7): Dropout(p=0.5, inplace=False)
  )
  (lstm): LSTM(256, 256, num_layers=4, batch_first=True, dropout=0.1, bidirectional=True)
  (classification): Sequential(
    (0): Linear(in_features=512, out_features=2048, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=2048, out_features=1024, bias=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=1024, out_features=41, bias=True)
  )
)


# Training Configuration (TODO)

In [24]:
import torch.optim as optim
criterion = nn.CTCLoss()
# TODO: What loss do you need for sequence to sequence models? 
# Do you need to transpose or permute the model output to find out the loss? Read its documentation
optimizer = optim.Adam(model.parameters(), lr = 2e-3)
# TODO: Adam works well with LSTM (use lr = 2e-3)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, threshold=0.0001, threshold_mode='abs', cooldown=0, min_lr=0.00005,)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, threshold=0.0001, threshold_mode='abs', cooldown=0, min_lr=0.00005,)
decoder = CTCBeamDecoder(
    G_PHONEME_MAP,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=10,
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)
# TODO: Intialize the CTC beam decoder
# Check out https://github.com/parlance/ctcdecode for the details on how to implement decoding
# Do you need to give log_probs_input = True or False?

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#WEIGHT_FILE = '/content/drive/MyDrive/IDL_Code_Model/HW2P2_Model/convnext_identity_epoch_79.pth'
WEIGHT_FILE = '/content/drive/MyDrive/HW2P3_Model/hw3p2_79.pth'
checkpoint = torch.load(WEIGHT_FILE, map_location=device)
epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['model_state_dict'])
criterion.load_state_dict(checkpoint['criterion_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

FileNotFoundError: ignored

In [25]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold=0.0001, threshold_mode='abs', cooldown=0, min_lr=0.00001,)


In [ ]:
epochs = 150
model.train()
pre_min = 0.17
#

for epoch in range(81, epochs):
    # Optional but recommended
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader, 0):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x = x.cuda()
        y = y.cuda()


        with torch.cuda.amp.autocast():     
            outputs, output_len = model(x,lx)
            loss = criterion(outputs.permute(1,0,2), y, output_len, ly)
        # Write a test code do perform a single forward pass and also compute the Levenshtein distance
        # Make sure that you are able to get this right before going on to the actual training
        # You may encounter a lot of shape errors
        # Printing out the shapes will help in debugging
        # Keep in mind that the Loss which you will use requires the input to be in a different format and the decoder expects it in a different format
        # Make sure to read the corresponding docs about it
        total_loss += float(loss)
        loss.backward()
        optimizer.step()

        batch_bar.set_postfix(loss="{:.07f}".format(float(total_loss / (i + 1))),
                              lr="{:.07f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()
    if total_loss / (i + 1) <= pre_min:
        pre_min = total_loss / (i + 1)
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
                'criterion_state_dict': criterion.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()},
                f'/content/drive/MyDrive/HW2P3_Model/hw3p2_{epoch}.pth', _use_new_zipfile_serialization = False)
    batch_bar.close()
    scheduler.step(total_loss)    


                #'scheduler_state_dict': scheduler.state_dict(),
                #'scaler_state_dict': scaler.state_dict()},
    
    

    print("Epoch {}/{}: Train Loss {:.07f}, Learning Rate {:.07f}".format(
          epoch + 1,
          epochs,
          float(total_loss / len(train_loader)),
          float(optimizer.param_groups[0]['lr'])))

Train:  52%|█████▏    | 231/446 [04:32<04:16,  1.19s/it, loss=3.4761238, lr=0.0020000]

In [ ]:
# this function calculates the Levenshtein distance 

def calculate_levenshtein(h, y, lh, ly, decoder, PHONEME_MAP):

    # h - ouput from the model. Probability distributions at each time step 
    # y - target output sequence - sequence of Long tensors
    # lh, ly - Lengths of output and target
    # decoder - decoder object which was initialized in the previous cell
    # PHONEME_MAP - maps output to a character to find the Levenshtein distance

    # TODO: You may need to transpose or permute h based on how you passed it to the criterion
    # Print out the shapes often to debug

    # TODO: call the decoder's decode method and get beam_results and out_len (Read the docs about the decode method's outputs)
    # Input to the decode method will be h and its lengths lh 
    # You need to pass lh for the 'seq_lens' parameter. This is not explicitly mentioned in the git repo of ctcdecode.
    beam_result, beam_scores, timesteps, out_lens = decoder.decode(h,lh)

    batch_size = int(beam_result.size(dim=0)) # TODO

    dist = 0

    for i in range(batch_size): # Loop through each element in the batch
        h_sliced = beam_result[i][0][:out_lens[i][0]]
        # TODO: Get the output as a sequence of numbers from beam_results
        # Remember that h is padded to the max sequence length and lh contains lengths of individual sequences
        # Same goes for beam_results and out_lens
        # You do not require the padded portion of beam_results - you need to slice it with out_lens 
        # If it is confusing, print out the shapes of all the variables and try to understand

        h_string = ''.join([PHONEME_MAP[i] for i in h_sliced])# TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string

        y_sliced = y[i][:ly[i]]
        # TODO: Do the same for y - slice off the padding with ly
        y_string = ''.join([PHONEME_MAP[i] for i in y_sliced])# TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string
        
        dist += Levenshtein.distance(h_string, y_string)

    dist/=batch_size

    return dist

decoder = CTCBeamDecoder(
G_PHONEME_MAP,
model_path=None,
alpha=0,
beta=0,
cutoff_top_n=40,
cutoff_prob=1.0,
beam_width=10,
num_processes=4,
blank_id=0,
log_probs_input=True
)

In [ ]:
total_dist = 0
model.eval()
for i, data in enumerate(val_loader, 0):
    optimizer.zero_grad()

    x, y, lx, ly = data
    x = x.cuda()
    y = y.cuda()

    outputs, output_len = model(x,lx)
    total_dist += calculate_levenshtein(outputs, y, output_len, ly, decoder, G_PHONEME_MAP)
    print(total_dist / (i + 1))

8.6875
8.2109375
7.645833333333333
7.51171875
7.325
7.434895833333333
7.399553571428571
7.52734375
7.710069444444445
7.709375
7.7258522727272725
7.743489583333333


KeyboardInterrupt: ignored

In [ ]:
torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()},
           f'/content/drive/MyDrive/IDL_Code_Model/HW2P2_Model/hw3p2_{epoch}.pth', _use_new_zipfile_serialization = False)

           #'scheduler_state_dict': scheduler.state_dict(),
            #'scaler_state_dict': scaler.state_dict()},

In [ ]:
# this function calculates the Levenshtein distance 

def find_beam(h, lh, decoder, PHONEME_MAP):

    beam_result, beam_scores, timesteps, out_lens = decoder.decode(h,lh)

    batch_size = int(beam_result.size(dim=0)) # TODO

    res = []


    for i in range(batch_size): # Loop through each element in the batch
        h_sliced = beam_result[i][0][:out_lens[i][0]]
        # TODO: Get the output as a sequence of numbers from beam_results
        # Remember that h is padded to the max sequence length and lh contains lengths of individual sequences
        # Same goes for beam_results and out_lens
        # You do not require the padded portion of beam_results - you need to slice it with out_lens 
        # If it is confusing, print out the shapes of all the variables and try to understand

        h_string = ''.join([PHONEME_MAP[i] for i in h_sliced])# TODO: MAP the sequence of numbers to its corresponding characters with PHONEME_MAP and merge everything as a single string

        res.append(h_string)
    return res

In [ ]:
model.eval()
batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')

res = []
for i, data in enumerate(test_loader, 0):
    x, lx = data

    # TODO: Finish predicting on the test set.
    x = x.cuda()

    with torch.no_grad():
        outputs,output_len = model(x, lx)
        res.extend(find_beam(outputs,output_len, decoder, G_PHONEME_MAP))

    batch_bar.update()
    
batch_bar.close()



In [ ]:
with open("early_submission.csv", "w+") as f:
    f.write("id,predictions\n")
    for i in range(len(res)):
        f.write("{},{}\n".format(i, res[i]))

In [ ]:
! kaggle competitions submit -c 11-785-s22-hw3p2 -f early_submission.csv -m early

100% 213k/213k [00:05<00:00, 43.1kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)

In [ ]:
torch.cuda.empty_cache() # Use this often

# TODO: Write the model evaluation function if you want to validate after every epoch

# You are free to write your own code for model evaluation or you can use the code from previous homeworks' starter notebooks
# However, you will have to make modifications because of the following.
# (1) The dataloader returns 4 items unlike 2 for hw2p2
# (2) The model forward returns 2 outputs
# (3) The loss may require transpose or permuting

# Note that when you give a higher beam width, decoding will take a longer time to get executed
# Therefore, it is recommended that you calculate only the val dataset's Levenshtein distance (train not recommended) with a small beam width
# When you are evaluating on your test set, you may have a higher beam width

In [ ]:
torch.cuda.empty_cache()
model.train()

# TODO: Write the model training code 

# You are free to write your own code for training or you can use the code from previous homeworks' starter notebooks
# However, you will have to make modifications because of the following.
# (1) The dataloader returns 4 items unlike 2 for hw2p2
# (2) The model forward returns 2 outputs
# (3) The loss may require transpose or permuting

# Tip: Implement mixed precision training

# Submit to kaggle (TODO)

In [ ]:
# TODO: Write your model evaluation code for the test dataset
# You can write your own code or use from the previous homewoks' stater notebooks
# You can't calculate loss here. Why?

In [ ]:
# TODO: Generate the csv file
! lsof /dev/nvidia*


In [ ]:
! kill -9 887